<a href="https://colab.research.google.com/github/Dkepffl/Dacon/blob/main/HD_AI_Challenge/HD_FE_1020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HD 현대 AI Chalenge**
- 구글 드라이브 어마운트를 통해 데이터 로드
- 리더보드 최저 점수(26.8103796658) 코드


## **| 구글 드라이브 어마운트**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **| 필요한 라이브러리 로드**

In [ ]:
# import libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os

## **| 작업 폴더 변경 및 데이터 로드**

In [ ]:
os.chdir("/content/drive/MyDrive/HD현대 AI Challenge") # 각자 드라이브에 맞춰 변경
os.getcwd()

'/content/drive/MyDrive/HD현대 AI Challenge'

In [ ]:
train = pd.read_csv('Data/train.csv').drop('SAMPLE_ID', axis=1)
test = pd.read_csv('Data/test.csv').drop('SAMPLE_ID', axis=1)

In [ ]:
# 원본 데이터셋 별도 저장
train_proto = train.copy()
test_proto = test.copy()

## **| 데이터 전처리**

### **1. ATA 변수 처리**

In [ ]:
from datetime import datetime

# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
  df['YEAR'] = df['ATA'].dt.year
  df['MONTH'] = df['ATA'].dt.month
  df['DAY'] = df['ATA'].dt.day
  df['HOUR'] = df['ATA'].dt.hour
  df['MINUTE'] = df['ATA'].dt.minute
  df['WEEKDAY'] = df['ATA'].dt.weekday

In [ ]:
train.drop('ATA', axis=1, inplace=True)
test.drop('ATA', axis=1, inplace=True)

In [ ]:
train.groupby('YEAR')['CI_HOUR'].agg(['mean','std','var'])

,mean,std,var
YEAR,,,
2014,95.657226,241.614838,58377.729818
2015,56.825993,168.889246,28523.577422
2016,53.570633,163.024751,26577.069383
2017,54.868054,156.360945,24448.745022
2018,60.376507,163.056411,26587.393033
2019,58.126869,167.055771,27907.630636
2020,63.534533,173.926701,30250.497449
2021,67.562373,175.861856,30927.392241
2022,70.726610,185.252965,34318.661030


### **2. 파생 변수 추가**

#### **선박 유형 별 선박 크기 관련 컬럼 추가**

In [ ]:
temp_train=train.copy()
temp_test=test.copy()

temp_train['SIZE']= temp_train['GT']+temp_train['BREADTH']+temp_train['DEADWEIGHT']+temp_train['DEPTH']+temp_train['DRAUGHT']+temp_train['LENGTH']
temp_test['SIZE']= temp_test['GT']+temp_test['BREADTH']+temp_test['DEADWEIGHT']+temp_test['DEPTH']+temp_test['DRAUGHT']+temp_test['LENGTH']

In [ ]:
ship_type1=train['SHIP_TYPE_CATEGORY'].unique().tolist()
ship_type2=test['SHIP_TYPE_CATEGORY'].unique().tolist()

In [ ]:
SHIP_dist_train=temp_train.groupby('SHIP_TYPE_CATEGORY')['SIZE'].agg(['mean','std']).reset_index()
SHIP_dist_test=temp_test.groupby('SHIP_TYPE_CATEGORY')['SIZE'].agg(['mean','std']).reset_index()

In [ ]:
for ship in ship_type1:
  train.loc[train['SHIP_TYPE_CATEGORY']==ship,'SIZE_MEAN'] = SHIP_dist_train.loc[SHIP_dist_train['SHIP_TYPE_CATEGORY']==ship,'mean'].values[0]
  train.loc[train['SHIP_TYPE_CATEGORY']==ship,'SIZE_STD'] = SHIP_dist_train.loc[SHIP_dist_train['SHIP_TYPE_CATEGORY']==ship,'std'].values[0]

for ship in ship_type2:
  test.loc[test['SHIP_TYPE_CATEGORY']==ship,'SIZE_MEAN'] = SHIP_dist_test.loc[SHIP_dist_test['SHIP_TYPE_CATEGORY']==ship,'mean'].values[0]
  test.loc[test['SHIP_TYPE_CATEGORY']==ship,'SIZE_STD'] = SHIP_dist_test.loc[SHIP_dist_test['SHIP_TYPE_CATEGORY']==ship,'std'].values[0]

#### **ATA_PO 별 DIST 통계량**

In [ ]:
port_list1=train['ARI_PO'].unique().tolist()
port_list2=test['ARI_PO'].unique().tolist()

In [ ]:
DIST_dist_train=train.groupby('ARI_PO')['DIST'].agg(['mean','std']).reset_index()
DIST_dist_test=train.groupby('ARI_PO')['DIST'].agg(['mean','std']).reset_index()

In [ ]:
for port in port_list1:
  train.loc[train['ARI_PO']==port,'DIST_MEAN'] = DIST_dist_train.loc[DIST_dist_train['ARI_PO']==port,'mean'].values[0]
  train.loc[train['ARI_PO']==port,'DIST_STD'] = DIST_dist_train.loc[DIST_dist_train['ARI_PO']==port,'std'].values[0]

for port in port_list2:
  test.loc[test['ARI_PO']==port,'DIST_MEAN'] = DIST_dist_test.loc[DIST_dist_test['ARI_PO']==port,'mean'].values[0]
  test.loc[test['ARI_PO']==port,'DIST_STD'] = DIST_dist_test.loc[DIST_dist_test['ARI_PO']==port,'std'].values[0]

#### **요일별 DIST 통계량**

In [ ]:
temp2_train=train.groupby('WEEKDAY')['DIST'].agg(['mean','std'])
temp2_test=train.groupby('WEEKDAY')['DIST'].agg(['mean','std'])

for i in range(0,7):
  train.loc[train['WEEKDAY']==i,'WEEK_DIST_MEAN'] = temp2_train.loc[temp2_train['WEEKDAY']==i,'mean'].values[0]
  train.loc[train['WEEKDAY']==i,'WEEK_DIST_STD'] = temp2_train.loc[temp2_train['WEEKDAY']==i,'std'].values[0]

for i in range(0,7):
  test.loc[test['WEEKDAY']==i,'WEEK_DIST_MEAN'] = temp2_test.loc[temp2_test['WEEKDAY']==i,'mean'].values[0]
  test.loc[test['WEEKDAY']==i,'WEEK_DIST_STD'] = temp2_test.loc[temp2_test['WEEKDAY']==i,'std'].values[0]

### **3. 카테고리 변수 라벨 인코딩**

In [ ]:
from sklearn.preprocessing import LabelEncoder
import bisect
from tqdm import tqdm

categorical_features = ['ARI_CO','ARI_PO', 'SHIP_TYPE_CATEGORY', 'ID', 'SHIPMANAGER','FLAG']
encoders = {}

for feature in tqdm(categorical_features, desc="Encoding features"):
  le = LabelEncoder()
  train[feature] = le.fit_transform(train[feature].astype(str))
  le_classes_set = set(le.classes_)
  test[feature] = test[feature].map(lambda s: '-1' if s not in le_classes_set else s)
  le_classes = le.classes_.tolist()
  bisect.insort_left(le_classes, '-1')
  le.classes_ = np.array(le_classes)
  test[feature] = le.transform(test[feature].astype(str))
  encoders[feature] = le

Encoding features: 100%|██████████| 6/6 [00:03<00:00,  1.87it/s]


### **4. 미사용 컬럼 제거**

In [ ]:
co_matrix=train.corr()
correlations_with_x = co_matrix['CI_HOUR']
correlations_with_x.abs().sort_values(ascending=False)

CI_HOUR               1.000000
DIST                  0.189145
WEEKDAY               0.111632
SHIP_TYPE_CATEGORY    0.087101
DEADWEIGHT            0.077954
GT                    0.059661
DEPTH                 0.051760
BREADTH               0.043711
DRAUGHT               0.040309
SIZE_MEAN             0.040032
AIR_TEMPERATURE       0.035827
LENGTH                0.035334
DIST_STD              0.028731
YEAR                  0.021253
FLAG                  0.021199
PORT_SIZE             0.020635
V_WIND                0.017415
BN                    0.016283
SIZE_STD              0.013462
U_WIND                0.012193
MINUTE                0.008505
ID                    0.008185
DIST_MEAN             0.007737
HOUR                  0.007474
ARI_CO                0.005224
MONTH                 0.004286
SHIPMANAGER           0.003544
BUILT                 0.001756
ARI_PO                0.000459
DAY                   0.000370
ATA_LT                0.000090
Name: CI_HOUR, dtype: float64

In [ ]:
train.drop(columns=['ID', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BUILT','BN'], inplace=True)
test.drop(columns=['ID', 'SHIPMANAGER', 'FLAG', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE', 'BUILT','BN'], inplace=True)

### **5. 결측치 처리**

In [ ]:
train = train.fillna(-1)
test = test.fillna(-1)

### **6. 스케일링**

In [ ]:
from sklearn.preprocessing import StandardScaler

scl = StandardScaler()

train_scld = scl.fit_transform(train.drop('CI_HOUR', axis=1))
train_scld = pd.DataFrame(train_scld, columns=train.drop('CI_HOUR', axis=1).columns)

test_scld = scl.transform(test)
test_scld = pd.DataFrame(test_scld, columns=test.columns)

In [ ]:
train_scld['CI_HOUR']=train['CI_HOUR']

## **| 모델 학습**

In [ ]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 93.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 107.8 MB/

In [ ]:
from pycaret.regression import *

In [ ]:
clf = setup(train_scld, target='CI_HOUR', train_size=0.8)

,Description,Value
0,Session id,4832
1,Target,CI_HOUR
2,Target type,Regression
3,Original data shape,"(391939, 23)"
4,Transformed data shape,"(391939, 23)"
5,Transformed train set shape,"(313551, 23)"
6,Transformed test set shape,"(78388, 23)"
7,Numeric features,22
8,Preprocess,True
9,Imputation type,simple


In [ ]:
comp = compare_models(sort='MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,55.3262,29190.7341,170.8245,-0.0083,1.7667,1.6841,4.0430
lightgbm,Light Gradient Boosting Machine,55.4445,20876.0685,144.4643,0.2787,1.3081,9.1045,6.9630
et,Extra Trees Regressor,55.6195,20813.6850,144.2451,0.2809,1.1533,9.0594,133.7000
xgboost,Extreme Gradient Boosting,55.9146,20786.6285,144.1611,0.2817,1.5418,7.8726,2.9250
rf,Random Forest Regressor,56.0200,20996.9288,144.8813,0.2746,1.1396,9.7514,250.4050
par,Passive Aggressive Regressor,59.2646,29106.1961,170.5617,-0.0051,2.0225,2.4038,0.9930
gbr,Gradient Boosting Regressor,61.0214,22630.6821,150.4122,0.2182,1.7757,9.6839,58.2840
knn,K Neighbors Regressor,64.7591,27148.4844,164.7469,0.0621,1.8595,5.4505,45.8900
dt,Decision Tree Regressor,70.1522,43074.2023,207.5199,-0.4896,1.2313,9.9052,3.9500
lasso,Lasso Regression,73.1559,26688.2605,163.3372,0.0782,2.5342,6.6751,0.5390


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
top1=create_model('huber',cross_validation=False)
top2=create_model('lightgbm',cross_validation=False)
top3=create_model('et',cross_validation=False)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Test,56.1676,30364.4829,174.2541,-0.0095,1.7720,1.8332


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Test,55.7831,21530.0780,146.7313,0.2842,1.3157,9.6605


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Test,55.6793,21217.6639,145.6628,0.2946,1.1453,8.6924


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
tuned_1 = tune_model(top1, optimize = 'MAE', n_iter = 10)
tuned_2 = tune_model(top2, optimize = 'MAE', n_iter = 10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,54.6797,28590.8465,169.0883,-0.0269,1.5715,1.3435
1,55.5260,29091.3302,170.5618,-0.0277,1.5794,1.3048
2,54.7099,28969.8606,170.2053,-0.0274,1.5684,1.2664
3,53.6570,27780.0083,166.6734,-0.0228,1.5732,1.3273
4,55.9205,31344.7539,177.0445,-0.0298,1.5810,1.2704
5,54.7771,29735.8293,172.4408,-0.0248,1.5683,1.3012
6,55.0040,30347.0492,174.2040,-0.0245,1.5671,1.3060
7,55.4572,31231.0968,176.7232,-0.0261,1.5684,1.4648
8,54.8232,29808.9327,172.6526,-0.0245,1.5787,1.2945


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,57.1275,20417.3265,142.8892,0.2667,1.5752,8.6220
1,57.7376,20592.1430,143.4996,0.2725,1.6539,9.1691
2,57.0140,21106.7565,145.2816,0.2515,1.5963,8.3675
3,55.9315,19748.5663,140.5296,0.2729,1.6242,8.3712
4,57.6876,21983.1128,148.2670,0.2778,1.6239,8.7554
5,57.2822,20935.1371,144.6898,0.2785,1.6146,9.3098
6,57.0180,21700.2443,147.3100,0.2674,1.6034,8.7267
7,57.2538,21230.6837,145.7075,0.3025,1.5793,9.2521
8,57.3219,21505.5749,146.6478,0.2609,1.6287,9.0928


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
tuned_3 = tune_model(top3, optimize = 'MAE', n_iter = 10)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:41:31
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
blender_specific = blend_models(estimator_list = [tuned_1, tuned_2, top3],optimize='MAE')

NameError: ignored

In [ ]:
final_model = finalize_model(blender_specific)
pred = predict_model(final_model, data=test_scld)

## **| 제출 파일 생성 및 저장**

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/HD현대 AI Challenge/Data/sample_submission.csv')
submit['CI_HOUR'] = pred

In [ ]:
submit.to_csv('YOUN/sub_FE1020(3).csv', index=False) # 파일 이름 수정